## Wallaroo API ML Engineer Example

Wallaroo provides the ability to perform inferences through deployed pipelines via the Wallaroo SDK and the Wallaroo MLOps API.  This tutorial demonstrates performing inferences using the Wallaroo MLOps API without being a named user


In [1]:
import json

import requests
from requests.auth import HTTPBasicAuth

import numpy as np
import pandas as pd

### Connect to Wallaroo

For this example, an Inference Token is used.  Unlike the Wallaroo MLOps API, this does **not** require a Wallaroo user account, only the confidential client and confidential client secret stored in the Wallaroo Keycloak service.  Typically the confidential client is `api-client`.

These details are loaded from the file `./data_engineer_creds.json`.

Update `wallarooPrefix = "YOUR PREFIX."` and `wallarooSuffix = "YOUR SUFFIX"` to match the Wallaroo instance used for this demonstration.  Note the `.` is part of the prefix.  If there is no prefix, then `wallarooPrefix = ""`

In [2]:
# Retrieve the login credentials.
f = open("./OCR_data_engineer_creds.json")

credentials = json.load(f)

wallarooPrefix = ""
wallarooSuffix = "wallaroo.realpage.com"

CONFIDENTIAL_CLIENT =credentials['secret_client']
CONFIDENTIAL_CLIENT_SECRET = credentials['secret_client_credential']
inference_url = credentials['inference_url']

In [3]:
def retrieve_inference_token(prefix, suffix, client, secret):
    tokenurl =f"https://{prefix}keycloak.{suffix}/auth/realms/master/protocol/openid-connect/token" 
    auth = HTTPBasicAuth(client, secret)
    data = {
    'grant_type': 'client_credentials'
    }
    response = requests.post(tokenurl, auth=auth, data=data, verify=True)
    print(response)
    return response.json()['access_token']

### Perform Inference Through External URL

The inference can now be performed through the External Inference URL.  This URL will accept the same inference data file that is used with the Wallaroo SDK, or with an Internal Inference URL as used in the Internal Pipeline Inference URL Tutorial.

For this example, the `externalUrl` retrieved through the [Get External Inference URL](#get-external-inference-url) 

Reference: [Wallaroo MLOps API Essentials Guide: Pipeline Management: Perform Inference Through External URL](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/wallaroo-mlops-api-essential-guide/wallaroo-mlops-api-essential-guide-pipelines/#perform-inference-through-external-url)

In [4]:
# set the headers
headers = {}
# Retrieve the token
token = retrieve_inference_token(wallarooPrefix, wallarooSuffix, CONFIDENTIAL_CLIENT, CONFIDENTIAL_CLIENT_SECRET)
headers['Authorization'] = f"Bearer {token}"
# set Content-Type type
headers['Content-Type']='application/json; format=pandas-records'



<Response [200]>


In [15]:
with open("sample_data/52807-combined.pdf", 'rb') as file:
    pdf_file_bytes = file.read()
    file.close()

with open("sample_data/k1.pdf",'rb') as file:
    pdf_file_bytes_1 = file.read()
    file.close()

buffered_input = np.frombuffer(pdf_file_bytes, dtype=np.uint8)
buffered_input_1 = np.frombuffer(pdf_file_bytes_1, dtype=np.uint8)

combined_df = pd.DataFrame({
        "input_product_code" : ["INV","K1"],
        "buffered_byte_files": [buffered_input.tolist(),buffered_input_1.tolist()]
    })

# For API
# Convert the DataFrame to a dictionary
input_dict = combined_df.to_dict(orient='records')


In [16]:
response = requests.post(
                inference_url, 
                json=input_dict, 
                headers=headers
                )

response

<Response [200]>

In [21]:
result = response.json()

print(f"Amount of results", len(result))

print("Amount of Images in First Result", len(result[0]["out"]["generated_text"]))

print(type(result[0]["out"]["generated_text"][0]))
print((result[0]["out"]["generated_text"][0]))



Amount of results 2
Amount of Images in First Result 2
<class 'str'>
{'INVOICE NUMBER': '52807', 'ACCOUNT NUMBER': 'LINCOLN PK', 'SALES #': None, 'DEPT. #': None, 'LOCATION': 'W', 'SOLD TO': 'RESIDENCES @ LINCOLN PK\n1 MIDWAY PARK DR\nN.DARTMOUTH\n(508)-938-5139', 'SHIP TO': 'SRV LINCOLN PK\n157580', 'INVOICE DATE': datetime.date(2024, 1, 11), 'PO NUMBER': '157580', 'TERMS': 'NET DUE', 'PAY METHOD': 'ON ACCOUNT', 'SUB TOTAL': 389.88, 'TAX': '14.99', 'DELIVERY': None, 'DISCOUNT': None, 'TOTAL': 404.87, 'LineItems': [{'QTY': '1', 'MAKE': 'LAB', 'PRODUCT': 'SVC', 'DESCRIPTION': 'SERVICE & DIAGNOSTIC', 'PRICE': '150.00', 'EXTENSION': '150.00'}, {'QTY': '1', 'MAKE': 'WCI', 'PRODUCT': '316405000', 'DESCRIPTION': 'SEAL', 'PRICE': '136.98', 'EXTENSION': '136.98'}, {'QTY': '1', 'MAKE': 'WCI', 'PRODUCT': '316575500', 'DESCRIPTION': 'BEZEL', 'PRICE': '102.90', 'EXTENSION': '102.90'}], 'Notes': '*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL\nALL OK NOW COMPLETE'}


In [20]:
result = response.json()

print(f"Amount of results", len(result))

print("Amount of Images in First Result", len(result[1]["out"]["generated_text"]))

print(type(result[1]["out"]["generated_text"][0]))
print((result[1]["out"]["generated_text"][0]))



Amount of results 2
Amount of Images in First Result 1
<class 'str'>
{'Schedule K-1 (Form 1065) 2021': {'Part I Information About the Partnership': {"A Partnership's employer identification number": '46-4097730', "B Partnership's name, address, city, state, and ZIP code": 'ENVIVA PARTNERS, LP\n7200 WISCONSIN AVENUE\nSUITE 1000\nBETHESDA, MD 20814', 'C IRS Center where partnership filed return': 'e-file', 'D Check if this is a publicly traded partnership (PTP)': 'BLANK'}, 'Part II Information About the Partner': {"E Partner's SSN or TIN (Do not use TIN of a disregarded entity. See instructions.)": '----7730', "F Partner's name, address, city, state, and ZIP code": 'SAMPLE K-1\n10000 UNITS\n7200 WISCONSIN AVENUE\nSUITE 1000\nBETHESDA, MD 20814', 'G General partner or LLC member-manager': 'BLANK', 'Limited partner or other LLC member': 'X', 'H1 Domestic partner': 'X', 'Foreign partner': 'BLANK', "H2 If the partner is a disregarded entity (DE), enter the partner's": {'TIN': 'BLANK', 'Name'

## Invoice

In [29]:
import ast
import re
import pandas as pd
pd.set_option('display.max_columns', None)

# Function to replace datetime.date with string dates
def replace_datetime_date(s):
    pattern = r'datetime\.date\((\d+), (\d+), (\d+)\)'
    def repl(match):
        year, month, day = match.groups()
        return f"'{year}-{month.zfill(2)}-{day.zfill(2)}'"
    return re.sub(pattern, repl, s)

# Process the data
processed_data = [replace_datetime_date(s) for s in result[0]["out"]["generated_text"]]

# Parse the strings into dictionaries
parsed_data = [ast.literal_eval(s) for s in processed_data]

# List to hold combined invoice and line item data
combined_data = []

for invoice in parsed_data:
    # Copy invoice data
    invoice_data = invoice.copy()
    
    # Standardize key names
    invoice_number = invoice_data.get('INVOICE NUMBER') or invoice_data.get('INVOICE')
    invoice_date = invoice_data.get('INVOICE DATE') or invoice_data.get('DATE')
    account_number = invoice_data.get('ACCOUNT NUMBER') or invoice_data.get('ACCT')
    sold_to = invoice_data.get('SOLD TO') or invoice_data.get('Sold To:')
    
    # Update the invoice data with standardized keys
    invoice_data['invoice_number'] = invoice_number
    invoice_data['invoice_date'] = invoice_date
    invoice_data['account_number'] = account_number
    invoice_data['sold_to'] = sold_to
    
    # Remove old keys
    for key in ['INVOICE NUMBER', 'INVOICE', 'INVOICE DATE', 'DATE', 'ACCOUNT NUMBER', 'ACCT', 'SOLD TO', 'Sold To:']:
        invoice_data.pop(key, None)
    
    # Determine line items key
    if 'LineItems' in invoice_data:
        line_items = invoice_data.pop('LineItems')
        # For each line item, combine with invoice data
        for item in line_items:
            combined_row = invoice_data.copy()
            combined_row.update(item)
            combined_data.append(combined_row)
    else:
        
        if 'QTY' in invoice_data:
            combined_row = invoice_data.copy()
            combined_data.append(combined_row)
        else:
            
            combined_data.append(invoice_data)


df = pd.DataFrame(combined_data)


df.rename(columns={
    'QTY': 'qty',
    'MAKE': 'make',
    'PRODUCT': 'product',
    'ITEM': 'product',
    'DESCRIPTION': 'description',
    'PRICE': 'price',
    'PRICE EA': 'price',
    'EXTENSION': 'extension',
    'TOTAL': 'total',
    'SUB TOTAL': 'subtotal',
    'SUBTOTAL': 'subtotal',
    'Tax': 'tax',
    'TAX': 'tax',
    'Notes': 'notes',
    'BALANCE': 'balance',
    'PAYMENTS': 'payments',
}, inplace=True)

df


,SALES #,DEPT. #,LOCATION,SHIP TO,PO NUMBER,TERMS,PAY METHOD,subtotal,tax,DELIVERY,DISCOUNT,total,notes,invoice_number,invoice_date,account_number,sold_to,qty,make,product,description,price,extension,EMPLID,product,price,subtotal,tax,payments,balance
0,NaN,NaN,W,SRV LINCOLN PK\n157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,52807,2024-01-11,LINCOLN PK,RESIDENCES @ LINCOLN PK\n1 MIDWAY PARK DR\nN.D...,1,LAB,SVC,SERVICE & DIAGNOSTIC,150.00,150.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,W,SRV LINCOLN PK\n157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,52807,2024-01-11,LINCOLN PK,RESIDENCES @ LINCOLN PK\n1 MIDWAY PARK DR\nN.D...,1,WCI,316405000,SEAL,136.98,136.98,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,W,SRV LINCOLN PK\n157580,157580,NET DUE,ON ACCOUNT,389.88,14.99,NaN,NaN,404.87,*APT 207*\nORDERED AND REPLACED SEAL AND BEZEL...,52807,2024-01-11,LINCOLN PK,RESIDENCES @ LINCOLN PK\n1 MIDWAY PARK DR\nN.D...,1,WCI,316575500,BEZEL,102.90,102.90,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,148.70,NaN,63170,2024-01-13,33477,MARION VILLAGE ESTATES\n32 VILLAGE DRIVE\nMARI...,1,NaN,NaN,PUSH BUTTON 1.1 CU.FT BLACK,NaN,NaN,LXHOUTMA,320023,139.95,139.95,8.75,148.70,148.70


## Inv2

In [5]:
with open("sample_data/91141.pdf", 'rb') as file:
    pdf_file_bytes = file.read()
    file.close()



inv_input = np.frombuffer(pdf_file_bytes, dtype=np.uint8)


inv_df = pd.DataFrame({
        "input_product_code" : ["INV"],
        "buffered_byte_files": [inv_input.tolist()]
    })

# For API
# Convert the DataFrame to a dictionary
input_dict = inv_df.to_dict(orient='records')


In [6]:
response = requests.post(
                inference_url, 
                json=input_dict, 
                headers=headers
                )

response

<Response [200]>

In [7]:
result = response.json()

print(f"Amount of results", len(result))

print("Amount of Images in First Result", len(result[0]["out"]["generated_text"]))

print(type(result[0]["out"]["generated_text"][0]))
print((result[0]["out"]["generated_text"][0]))



Amount of results 1
Amount of Images in First Result 1
<class 'str'>
{'invoice_number': '9114', 'po_number': 'CS/B', 'date': '1/14/24', 'vendor': 'A-PLUS DRAINS LLC', 'vendor_address': '51 Willow St, Unit 2386\nLynn, MA 01903', 'vendor_phone': '(781) 477-1587', 'vendor_fax': '(781) 477-1841', 'customer_name': 'Bluslm- United Estate', 'customer_address': '3 Nazin #8\nDorchester, MA', 'line_items': [], 'subtotal': None, 'tax': None, 'total_amount_due': 495.0, 'description_of_work': "Snaked main stack line for kitchen sinks 50' mutiple\ntimes to remove heavy grease and food throughat thr vanit.\nsnaked unit sink to remove buildup due to main line\nblockage Frohed all alt-running well", 'notes': 'A-PLUS DRAINS LLC DRAIN CLEANING SERVICE intentions are to unclog drains to allow water to flow by electric shaking/jetting of lines. Camera\nInspection of lines after snaking helps to assure pipe is clear from debris or objects, fatigue, scale, cans, toys, etc, which can cause damage with\nfuture

In [10]:
import ast
import re
import pandas as pd
pd.set_option('display.max_columns', None)

# Function to replace datetime.date with string dates
def replace_datetime_date(s):
    pattern = r'datetime\.date\((\d+), (\d+), (\d+)\)'
    def repl(match):
        year, month, day = match.groups()
        return f"'{year}-{month.zfill(2)}-{day.zfill(2)}'"
    return re.sub(pattern, repl, s)

# Process the data
processed_data = [replace_datetime_date(s) for s in result[0]["out"]["generated_text"]]

# Parse the strings into dictionaries
parsed_data = [ast.literal_eval(s) for s in processed_data]

combined_data = []

for invoice in parsed_data:
    invoice_data = invoice.copy()
    
    invoice_number = invoice_data.get('INVOICE NUMBER') or invoice_data.get('INVOICE')
    #invoice_date = invoice_data.get('INVOICE DATE') or invoice_data.get('DATE')
    account_number = invoice_data.get('ACCOUNT NUMBER') or invoice_data.get('ACCT')
    sold_to = invoice_data.get('SOLD TO') or invoice_data.get('Sold To:')
    
    invoice_data['invoice_number'] = invoice_number
    #invoice_data['invoice_date'] = invoice_date
    #invoice_data['account_number'] = account_number
    #invoice_data['sold_to'] = sold_to
    
    # Remove old keys
    for key in ['INVOICE NUMBER', 'INVOICE',  'DATE', 'ACCT']:
        invoice_data.pop(key, None)
    
    # Determine line items key
    if 'LineItems' in invoice_data:
        line_items = invoice_data.pop('LineItems')
        for item in line_items:
            combined_row = invoice_data.copy()
            combined_row.update(item)
            combined_data.append(combined_row)
    else:
        # No 'LineItems' key, check if line items are at the invoice level
        # We can check if 'QTY' exists in invoice_data
        if 'QTY' in invoice_data:
            combined_row = invoice_data.copy()
            combined_data.append(combined_row)
        else:
            # No line items, just add the invoice data
            combined_data.append(invoice_data)

df = pd.DataFrame(combined_data)


df


,invoice_number,po_number,date,vendor,vendor_address,vendor_phone,vendor_fax,customer_name,customer_address,line_items,subtotal,tax,total_amount_due,description_of_work,notes
0,None,CS/B,1/14/24,A-PLUS DRAINS LLC,"51 Willow St, Unit 2386\nLynn, MA 01903",(781) 477-1587,(781) 477-1841,Bluslm- United Estate,"3 Nazin #8\nDorchester, MA",[],None,None,495.0,Snaked main stack line for kitchen sinks 50' m...,A-PLUS DRAINS LLC DRAIN CLEANING SERVICE inten...


## K1

In [32]:
import ast
import pandas as pd
import numpy as np
import re

def parse_and_clean_data(data):
    parsed_data = [ast.literal_eval(entry) for entry in data]
    
    def flatten_dict(d, parent_key='', sep='_'):
        items = []
        for k, v in d.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            if isinstance(v, dict):
                items.extend(flatten_dict(v, new_key, sep=sep).items())
            else:
                items.append((new_key, v))
        return dict(items)
    
    # Flatten
    flattened_data = []
    for entry in parsed_data:
        for key, value in entry.items():
            flattened_entry = flatten_dict(value)
            flattened_entry['Form'] = key
            flattened_data.append(flattened_entry)
    
    df = pd.DataFrame(flattened_data)
    
    df.replace(['BLANK', ''], np.nan, inplace=True)
    
    df = df.applymap(lambda x: x.strip('$').replace(',', '') if isinstance(x, str) else x)
    df = df.applymap(lambda x: re.sub(r'\(([^)]+)\)', r'-\1', x) if isinstance(x, str) else x)
    
    numeric_cols = [
        "A Partnership's employer identification number",
        'Beginning capital account',
        'Capital contributed during the year',
        'Current year net income (loss)',
        'Other increase (decrease) (attach explanation)',
        'Withdrawals & distributions',
        'Ending capital account',
        '1 Ordinary business income (loss)',
        '5 Interest income',
        '6a Ordinary dividends',
        '9a Net long-term capital gain (loss)',
        '14 Self-employment earnings (loss)',
        '17 Alternative minimum tax (AMT) items',
        '18 Tax-exempt income and nondeductible expenses',
        '19 Distributions',
        '20 Other information_A',
        '20 Other information_V',
        '20 Other information_AA',
    ]
    
    for col in numeric_cols:
        if col in df.columns:
            df[col] = df[col].astype(str)
            df[col] = df[col].str.replace(' ', '') 
            df[col] = df[col].str.replace('L\*', '') 
            df[col] = df[col].replace('', np.nan)  
            df[col] = pd.to_numeric(df[col], errors='coerce') 
    
    percentage_cols = [
        'Profit_Beginning', 'Profit_Ending',
        'Loss_Beginning', 'Loss_Ending',
        'Capital_Beginning', 'Capital_Ending'
    ]
    for col in percentage_cols:
        if col in df.columns:
            df[col] = df[col].str.replace('%', '').str.strip()
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

df = parse_and_clean_data(result[1]["out"]["generated_text"])

df


,Part I Information About the Partnership_A Partnership's employer identification number,"Part I Information About the Partnership_B Partnership's name, address, city, state, and ZIP code",Part I Information About the Partnership_C IRS Center where partnership filed return,Part I Information About the Partnership_D Check if this is a publicly traded partnership (PTP),Part II Information About the Partner_E Partner's SSN or TIN (Do not use TIN of a disregarded entity. See instructions.),"Part II Information About the Partner_F Partner's name, address, city, state, and ZIP code",Part II Information About the Partner_G General partner or LLC member-manager,Part II Information About the Partner_Limited partner or other LLC member,Part II Information About the Partner_H1 Domestic partner,Part II Information About the Partner_Foreign partner,"Part II Information About the Partner_H2 If the partner is a disregarded entity (DE), enter the partner's_TIN","Part II Information About the Partner_H2 If the partner is a disregarded entity (DE), enter the partner's_Name",Part II Information About the Partner_I1 What type of entity is this partner?,"Part II Information About the Partner_I2 If this partner is a retirement plan (IRA/SEP/Keogh/etc.), check here","J Partner's share of profit, loss, and capital_Profit_Beginning","J Partner's share of profit, loss, and capital_Profit_Ending","J Partner's share of profit, loss, and capital_Loss_Beginning","J Partner's share of profit, loss, and capital_Loss_Ending","J Partner's share of profit, loss, and capital_Capital_Beginning","J Partner's share of profit, loss, and capital_Capital_Ending",K Partner's share of liabilities_Nonrecourse_Beginning,K Partner's share of liabilities_Nonrecourse_Ending,K Partner's share of liabilities_Qualified nonrecourse financing_Beginning,K Partner's share of liabilities_Qualified nonrecourse financing_Ending,K Partner's share of liabilities_Recourse_Beginning,K Partner's share of liabilities_Recourse_Ending,L Partner's capital account analysis_Beginning capital account,L Partner's capital account analysis_Capital contributed during the year,L Partner's capital account analysis_Current year net income (loss),L Partner's capital account analysis_Other increase (decrease) (attach explanation),L Partner's capital account analysis_Withdrawals & distributions,L Partner's capital account analysis_Ending capital account,M Did the partner contribute property with a built-in gain or loss?,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Beginning,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Ending,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Final K-1,N Partner's Share of Net Unrecognized Section 704(c) Gain or (Loss)_Amended K-1,"Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_1 Ordinary business income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_2 Net rental real estate income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_3 Other net rental income (loss)","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4a Guaranteed payments for services","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4b Guaranteed payments for capital","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_4c Total guaranteed payments","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_5 Interest income","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6a Ordinary dividends","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6b Qualified dividends","Part III Partner's Share of Current Year Income, Deductions, Credits, and Other Items_6c Dividend equivalents","Part III Partner's Share of Cu